In [ ]:
!pip install gdown

In [ ]:
import gdown
url = 'https://drive.google.com/uc?id=12vGmIScPzW2MWdA4Q_L4Iq9Uj-7Wpfsk'
output = 'dataset'
gdown.download(url, output)

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import numpy as np
import os
import random
import matplotlib.pyplot as plt

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
dataset = pd.read_csv('dataset')

In [ ]:
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()

In [ ]:
inspect_dataframe(dataset, dataset.columns)

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
train_size = int(dataset.shape[0] * 0.85)
X_train = dataset.iloc[:train_size]
X_test  = dataset.iloc[train_size:]
X_train.shape, X_test.shape

In [ ]:
X_min = X_train.min()
X_max = X_train.max()

X_train = (X_train - X_min)/(X_max - X_min)
X_test = (X_test - X_min)/(X_max - X_min)

In [ ]:
inspect_dataframe(X_train, X_train.columns)

In [ ]:
def build_sequences(df, target_labels, window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset     = []
    labels      = []
    temp_df     = df.copy().values
    temp_label  = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding     = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df     = np.concatenate((padding,df))
        padding     = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label  = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels  = np.array(labels)
    return dataset, labels

In [ ]:
target_labels = dataset.columns
window = 200
stride = 10
telescope = 50
X_train, y_train = build_sequences(X_train,target_labels, window, stride, telescope)
X_test , y_test  = build_sequences(X_test, target_labels, window, stride, telescope)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

#     convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(input_layer)
#     convlstm = tfkl.GRU(128, activation='tanh', dropout=0.6, recurrent_dropout=0.6, return_sequences=True)(input_layer)
    convlstm = tfkl.Conv1D(128, 3, padding='same', activation='relu')(input_layer)
    convlstm = tfkl.Dropout(.5)(convlstm)
    convlstm = tfkl.Conv1D(256, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.Dropout(.5)(convlstm)
    convlstm = tfkl.MaxPool1D()(convlstm)
#     convlstm = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True))(convlstm)
#     convlstm = tfkl.GRU(128, activation='tanh', dropout=0.6, recurrent_dropout=0.6, return_sequences=True)(convlstm)
    convlstm = tfkl.Conv1D(512, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5)(convlstm)

    # In order to predict the next values for more than one channel,
    # we can use a Dense layer with a number given by telescope*num_channels,
    # followed by a Reshape layer to obtain a tensor of dimension 
    # [None, telescope, num_channels]
    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu')(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])

    # Return the model
    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
# tfk.utils.plot_model(model, expand_nested=True)

In [ ]:
epochs     = 200
batch_size = 24
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

In [ ]:
plt.plot(history['val_loss'], label='vloss')
plt.plot(history['loss'], label='loss')
plt.legend(loc='lower right')
plt.show()

In [ ]:
predictions = model.predict(X_test)
print(predictions.shape)

rmse = tfk.metrics.RootMeanSquaredError()
rmse.update_state(y_test, predictions)
rmse.result()

In [ ]:
model.save('only_conv1D_model')

In [ ]:
import shutil
shutil.make_archive("only_conv1D_model", "zip", "only_conv1D_model")

In [ ]:
from IPython.display import FileLink
FileLink('only_conv1D_model.zip')